<img src="https://digital.hbs.edu/platform-digit/wp-content/uploads/sites/2/2021/03/mercado-libre-logo-2.png" alt="drawing" width="300" />

# [MEN-148](https://mercadolibre.atlassian.net/browse/MEN-148) - Mejoras RNN - Versión 1.0

## 2. Desarrollo

In [6]:
# estos primeros 3 installs son para la visualizar la arquitectura de la red neuronal
# !pip install pydot
# !pip install pydotplus
# !apt-get install graphviz

import warnings
warnings.filterwarnings('ignore')

from commons.db import BigQueryConn
import dask.dataframe as dd
# from dask.dataframe import from_pandas
import pandas as pd
import itertools
import json
import re
from sklearn.utils import class_weight
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dropout

from commons.utils.timer import timer

pd.set_option('display.max_colwidth', None)
# import seaborn as sns
# import matplotlib.pyplot as plt

# Vuelvo con el approach anterior con compras

In [7]:
import os
import google_cloud
import base64

AUTH_BIGQUERY = os.environ["SECRET_MLD_MKETING_GCP_CRED"]
bq = google_cloud.BigQuery(AUTH_BIGQUERY)  #conectate

In [87]:
bq.fast_export("""
    WITH VISITAS AS(
        SELECT a11.ORD_BUYER.ID as CUS_CUST_ID , sit_site_id, ORD_CREATED_DT as ord_date, ORD_CLOSED_DT as closed_dt
        FROM `meli-bi-data.WHOWNER.BT_ORD_ORDERS` a11 
        INNER JOIN mld-marketing.clustering.catalog_categories catalog
            ON CAST(SUBSTR(catalog.category_id ,4)as INT64) = a11.ORD_CATEGORY.ID
            AND a11.SIT_SITE_ID = catalog.site_id
        INNER JOIN mld-marketing.clustering.catalog_categories_cleaned catalog_clean
            ON catalog.domain_id = catalog_clean.domain_id
            AND catalog.site_id = catalog_clean.site_id
            AND catalog.domain_id IS NOT NULL 
        WHERE a11.ORD_STATUS = 'paid' AND a11.SIT_SITE_ID = 'MLA'
            AND a11.ORD_CLOSED_DT in (date'2021-08-01',date'2021-07-31',date'2021-07-30')
        GROUP BY 1,2,3,4
    ),
    
    TRAIN_FEATURES AS(
        SELECT  event.cus_cust_id, event.sit_site_id,closed_dt,
                ARRAY_REVERSE(
                    --ordena los 53 eventos mas recientes encontrados, con la fecha mas antigua primero
                    ARRAY_AGG(
                        --captura los 53 eventos mas recientes del usuario
                        STRUCT(
                            DATE_DIFF(DATE_ADD(ord_date, INTERVAL 1 DAY), event.log_date, DAY) as days_diff,
                            catalog_clean.token as token,
                            CASE WHEN 
                                (VIP_VIEW_APP_ANDROID_ITEM_COUNT + VIP_VIEW_APP_IOS_ITEM_COUNT + VIP_VIEW_DESKTOP_ITEM_COUNT + VIP_VIEW_WEB_MOBILE_ITEM_COUNT) > 5 THEN 3
                            WHEN
                                (VIP_VIEW_APP_ANDROID_ITEM_COUNT + VIP_VIEW_APP_IOS_ITEM_COUNT + VIP_VIEW_DESKTOP_ITEM_COUNT + VIP_VIEW_WEB_MOBILE_ITEM_COUNT) = 1 THEN 1
                            ELSE
                                2
                            END as vips)
                        ORDER BY event.log_date desc
                        LIMIT 53
                        )
                ) as train_events
        FROM meli-marketing.MODELLING.LK_EVENT_CUSTOMER_ITEMS event 
        INNER JOIN VISITAS v
            ON v.CUS_CUST_ID = event.CUS_CUST_ID
            AND v.SIT_SITE_ID = event.SIT_SITE_ID
        INNER JOIN mld-marketing.clustering.catalog_categories catalog
            ON CAST(SUBSTR(catalog.category_id ,4)as INT64) = event.cat_categ_id
            AND event.SIT_SITE_ID = catalog.site_id
            AND catalog.domain_id IS NOT NULL 
        INNER JOIN mld-marketing.clustering.catalog_categories_cleaned catalog_clean
            ON catalog.domain_id = catalog_clean.domain_id
            AND catalog.site_id = catalog_clean.site_id
        WHERE  (VIP_VIEW_APP_ANDROID_ITEM_COUNT + VIP_VIEW_APP_IOS_ITEM_COUNT + VIP_VIEW_DESKTOP_ITEM_COUNT + VIP_VIEW_WEB_MOBILE_ITEM_COUNT) > 0
                AND log_date < ord_date AND log_date >= DATE_SUB(ord_date, INTERVAL 60 DAY)
        GROUP BY 1,2,3
    ),
    TARGET_FEATURES AS(
        SELECT catalog_clean.token as target_token, a11.ORD_BUYER.ID as CUS_CUST_ID,
                a11.sit_site_id,closed_dt
        FROM `meli-bi-data.WHOWNER.BT_ORD_ORDERS` a11 
        INNER JOIN VISITAS v
            ON v.CUS_CUST_ID = a11.ORD_BUYER.ID
            AND v.SIT_SITE_ID = a11.SIT_SITE_ID
            AND a11.ORD_CLOSED_DT = v.closed_dt
        INNER JOIN mld-marketing.clustering.catalog_categories catalog
            ON CAST(SUBSTR(catalog.category_id ,4)as INT64) = a11.ORD_CATEGORY.ID
            AND a11.SIT_SITE_ID = catalog.site_id
        INNER JOIN mld-marketing.clustering.catalog_categories_cleaned catalog_clean
            ON catalog.domain_id = catalog_clean.domain_id
            AND catalog.site_id = catalog_clean.site_id
            AND catalog.domain_id IS NOT NULL 
        WHERE a11.ORD_STATUS = 'paid' AND a11.SIT_SITE_ID = 'MLA'
            
        GROUP BY 1,2,3,4
    )
    SELECT train.cus_cust_id,train_events,target_token
    FROM TRAIN_FEATURES train
    INNER JOIN TARGET_FEATURES target
        on target.cus_cust_id = train.cus_cust_id
        AND target.sit_site_id = train.sit_site_id
        AND train.closed_dt = target.closed_dt
    where train.sit_site_id = 'MLA'
        -- and MOD(train.cus_cust_id,100) < 4
""","gs://ext_ngrasso_fda/temp/training_rnn_next_purchase",temp_dataset="clustering")

Query results loaded to the table mld-marketing.clustering.ponspuhedt
Exporto mld-marketing:clustering.ponspuhedt a gs://ext_ngrasso_fda/temp/training_rnn_next_item/*.parquet
Deleted table 'mld-marketing.clustering.ponspuhedt'.


In [8]:
# traemos la cantidad de dominios en MLA
site_id="MLA"
clean_domains_query = f"""
    SELECT  domain_id,token
    FROM `mld-marketing.clustering.catalog_categories_cleaned` 
    WHERE site_id = '{site_id}'
"""
domains = bq.execute_response(clean_domains_query)

In [18]:
# Read files from storage and build a lazy dask dataframe to later get list of dicts
destination = 'gs://ext_ngrasso_fda/temp/training_rnn_next_purchase/*.parquet' #todo MLA con token categories_cleaned, truncado 54 eventos
df = dd.read_parquet(destination, storage_options={"token": json.loads(os.environ["SECRET_MLD_MKETING_GCP_CRED"])}).compute()

In [19]:
domains_amt = df.target_token.value_counts()
df = df.loc[df.target_token.isin(domains_amt[domains_amt>=10].index)]

In [21]:
conversor = df[["target_token"]].drop_duplicates()
conversor["token_categ"] = conversor['target_token'].astype("category").cat.codes + 1
mapper = {x["target_token"]:x["token_categ"] for _,x in conversor.iterrows() }

In [22]:
import random

max_len = 53

def two_steps(events):
    """
    Separamos los dominios de cada usuario en forma de secuencia de caracteres separado por coma,
    algo similar para los days_diff pero en forma de lista
    """
    tokens = []
    days = []
    vips = []  
    append_tokens = tokens.append
    append_days = days.append
    append_vips = vips.append
    
    for event in events:
        if(event['token'] in mapper):
            append_tokens(mapper[event['token']])
            append_days(event['days_diff'])
            append_vips(event['vips'])
    return [tokens, days,vips]

#recuperamos los dominios y dias asociados a cada visita
df['raw_sequence'] = df['train_events'].apply(lambda x: two_steps(x))
df["input_domains"] = df["raw_sequence"].apply(lambda x: x[0])
df["input_days"] = df["raw_sequence"].apply(lambda x: x[1])
df["input_vips"] = df["raw_sequence"].apply(lambda x: x[2])

df["n_domains"] = df["input_domains"].apply(lambda x: len(x))
df = df.loc[df.n_domains > 4]

del df['raw_sequence']

df["input_domains"] = df["input_domains"].apply(lambda x: pad_sequences([x], maxlen=max_len)[0])
df["input_days"] = df["input_days"].apply(lambda x: pad_sequences([x], maxlen=max_len)[0])
df["input_vips"] = df["input_vips"].apply(lambda x: pad_sequences([x], maxlen=max_len)[0])

df["target_token"] = df["target_token"].apply(lambda x: mapper[x])

In [24]:
# df.to_pickle("to_rnn.pkl")
# # !pip install google-cloud-core==1.5.0
# import os
# import google_cloud
# import base64

# AUTH_BIGQUERY = os.environ["SECRET_MLD_MKETING_GCP_CRED"]
# st = google_cloud.Storage(AUTH_BIGQUERY)  #conectate

# st.upload_file("to_rnn.pkl","gs://ext_ngrasso_fda/temp/training_dataset_sample_rnn_compras.pkl")

In [14]:
# df.to_pickle("to_rnn.pkl")
# !pip install google-cloud-core==1.5.0
import os
import google_cloud
import base64

AUTH_BIGQUERY = os.environ["SECRET_MLD_MKETING_GCP_CRED"]
st = google_cloud.Storage(AUTH_BIGQUERY)  #conectate

st.downlad_file("gs://ext_ngrasso_fda/temp/training_dataset_sample_rnn_compras.pkl","rnn.pkl")
df = pd.read_pickle("rnn.pkl")

max_len = 53
vocab_size = df.target_token.nunique() + 1 # el 0 tambien cuenta

In [25]:
# Splitting for train, validation
df_train, _ = train_test_split(df, test_size = 0.4, random_state = 0, stratify=df.target_token)
df_train, df_validation = train_test_split(df_train, test_size = 0.2, random_state = 0, stratify=df_train.target_token)

In [26]:
del df

In [27]:
#pasamos de numpy array a dict
class_weights = class_weight.compute_class_weight('balanced', np.unique(df_train.target_token), df_train.target_token)
weights = {i+1: class_weights[i] for i in range(len(class_weights))}
weights[0] = 0

In [28]:
# pasamos los datos de input a tensores para que el modelo pueda usarlo.
X_seq_domains_train = tf.convert_to_tensor(list(df_train.input_domains))
X_seq_days_train =  tf.convert_to_tensor(list(df_train.input_days))
X_seq_vips_train =  tf.convert_to_tensor(list(df_train.input_vips))
y_train_train = tf.convert_to_tensor(list(df_train.target_token))

# X_seq_domains_test = tf.convert_to_tensor(list(df_test.input_domains))
# X_seq_days_test =  tf.convert_to_tensor(list(df_test.input_days))
# X_seq_vips_test =  tf.convert_to_tensor(list(df_test.input_vips))
# y_train_test = tf.convert_to_tensor(list(df_test.target_token))

X_seq_domains_validation = tf.convert_to_tensor(list(df_validation.input_domains))
X_seq_days_validation =  tf.convert_to_tensor(list(df_validation.input_days))
X_seq_vips_validation =  tf.convert_to_tensor(list(df_validation.input_vips))
y_train_validation = tf.convert_to_tensor(list(df_validation.target_token))


vocab_size_days = np.array(X_seq_days_train).max() + 1

2022-02-08 09:12:31.584092: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-08 09:12:31.584133: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-08 09:12:31.584161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (5ffb8fcf4aa7): /proc/driver/nvidia/version does not exist
2022-02-08 09:12:31.584657: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-08 09:12:31.614203: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 240000000

In [29]:
def loss_function(real, pred):
    """
    We redefine our own loss function in order to get rid of the '0' value
    which is the one used for padding. This to avoid that the model optimize itself
    by predicting this value because it is the padding one.
    
    :param real: the truth
    :param pred: predictions
    :return: a masked loss where '0' in real (due to padding)
                are not taken into account for the evaluation
    """

    # to check that pred is numric and not nan
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_object_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                 reduction='none')
    loss_ = loss_object_(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                   mode='min', 
                   patience=5, 
                   verbose=1, 
                   restore_best_weights=True)

In [30]:
# l1 = tf.keras.layers.Embedding(3, # numeros distintos
#                           4, # dimension del vector
#                           input_length=6 # cantidad de datos de entrada
#                          )(np.array([[0,1,2,1,2,0],
#                                     [0,1,2,1,2,0]]))
# l2 = tf.constant(np.array([[10.,100.,1000.,10000.,100000.,100000.],
#                           [10.,100.,1000.,10000.,100000.,100000.]]))
# l2 = tf.keras.layers.Reshape((6,1))(l2)
# tf.keras.layers.multiply([l1,l2]),l1

In [31]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

#reset Keras Session
def reset_keras(model):
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
from sklearn.model_selection import ParameterSampler
from keras import backend as K
# from numba import cuda

# me armo el set de testeo para ver novelty y covertura
test_validation = df_validation.sample(frac=1.).drop_duplicates(subset="cus_cust_id")
X_seq_domains_test_val = tf.convert_to_tensor(list(test_validation.input_domains))
X_seq_days_test_val =  tf.convert_to_tensor(list(test_validation.input_days))
X_seq_vips_test_val =  tf.convert_to_tensor(list(test_validation.input_vips))


def train_model_simil_mult(layers,nn1,lstm,lr,l1,l2,w_scaler = None,emb_domain=100,emb_days=100):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''

    drop_out = 0.5
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
    adam = keras.optimizers.Adam(lr=lr)
    
    input_domain = keras.Input(shape=(X_seq_domains_train.shape[1],), name='input_dominios')
    embedding_domain = keras.layers.Embedding(vocab_size, int(emb_domain), input_length=max_len)(input_domain)
    
    input_mult = keras.Input(shape=(X_seq_vips_train.shape[1],), name='input_dmultipliersVip')
    input_T_mult = tf.keras.layers.Reshape((X_seq_vips_train.shape[1],1))(input_mult)
    embedding_domain_mult = tf.keras.layers.multiply([embedding_domain,input_T_mult])
    embedding_domain_mult = keras.layers.BatchNormalization(name='batchnorm_embedding')(embedding_domain_mult)
    
    encoding_padding_mask = tf.math.logical_not(tf.math.equal(input_domain, 0)) # para que no le de bola al padding
    
    input_days = keras.Input(shape=(X_seq_days_train.shape[1],),name='nb_days')    
    embedding_days = keras.layers.Embedding(vocab_size_days, int(emb_days), input_length=max_len)(input_days)
    
    concat = keras.layers.concatenate([embedding_domain_mult, embedding_days], axis=-1,name='concat_embedding_input')
    
    rnn = keras.layers.LSTM(units=lstm,return_sequences=True,stateful=False,
               recurrent_initializer='glorot_normal',name='LSTM_cat'
              )(concat)
    
    rnn = keras.layers.BatchNormalization(name='batchnorm_lstm')(rnn)
    
    att = keras.layers.Attention(use_scale=False, causal=True,
                                    name='attention')(inputs=[rnn, rnn],
                                                      mask=[encoding_padding_mask,
                                                            encoding_padding_mask])
    
    att_flatt = keras.layers.Flatten()(att)
#     concat_features = keras.layers.concatenate([att_flatt, input_metadata,flatt_most_vist], axis=-1,name='concat_features')
    dense1 = keras.layers.Dense(nn1,'relu',
                                kernel_regularizer=reg ,
                                name='dense1')(att_flatt)
    dense1 = Dropout(drop_out)(dense1)
    dense1 = keras.layers.BatchNormalization(name='batchnorm_lstm1')(dense1)
    
    if (layers == 2):
        dense1 = keras.layers.Dense(nn2,'relu',
                                kernel_regularizer=reg ,
                                name='dense2')(dense1)
        dense1 = Dropout(drop_out)(dense1)
        dense1 = keras.layers.BatchNormalization(name='batchnorm_lstm2')(dense1)
        
    if (layers == 3):
        dense1 = keras.layers.Dense(nn3,'relu',
                            kernel_regularizer=reg ,
                            name='dense3')(dense1)
        dense1 = Dropout(drop_out)(dense1)
        dense1 = keras.layers.BatchNormalization(name='batchnorm_lstm3')(dense1)
        
    output = tf.keras.layers.Dense(vocab_size, name='output', activation='softmax')(dense1)

    model = keras.Model([input_domain,input_mult,input_days], output)

    model.compile(loss=loss_function, optimizer=adam, metrics=["sparse_categorical_accuracy"])
    
    return model

layers = [1]
nn1 = [3072,4096,5000,6144,7000,8000,9000]
# nn2 = [100,512,1024,2048,3072,4096,5120]
lstm = [50,75,100]
w_scaler = [1.5,2,2.2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]

# dropout and regularisation
l1 = [0.1,0.01, 0.001]
l2 = [0, 0.1,0.01, 0.001]
lr = [10**-3]


# dictionary summary
param_grid = dict(
                    layers= layers,nn1=nn1,lstm=lstm, l1=l1, l2=l1, lr = lr,w_scaler = w_scaler              
                 )

i = 1
for a in [len(x) for x in param_grid.values() if(type(x) != int)]: i=a*i
print("Cantidad de parametros:",i)

rng = np.random.RandomState(0)
param_list = list(ParameterSampler(param_grid, n_iter=100,
                                   random_state=rng))

import csv  
# import recmetrics

l_hist = []
for params in param_list:
    print(params)
    model = train_model_simil_mult(**params)
    
    weights = {i+1: class_weights[i]**params["w_scaler"] for i in range(len(class_weights))}
    weights[0] = 0

    hist = model.fit([X_seq_domains_train,X_seq_vips_train,X_seq_days_train], y_train_train, 
                 validation_data = ([X_seq_domains_validation, X_seq_vips_validation,X_seq_days_validation], y_train_validation),
                 epochs = 100, batch_size=512, callbacks=[early_stopping],
                 class_weight = weights)

    coverage = np.unique(np.argsort(-model.predict([X_seq_domains_test_val, X_seq_vips_test_val,X_seq_days_test_val]))[:,0:5]).shape[0]/3060

    with open("rnn_train_metrics.csv", 'a') as f:
        writer = csv.writer(f)
        writer.writerow([params,coverage,max(hist.history["val_sparse_categorical_accuracy"])])
        
#     #limpio la memoria de la GPU para que corra todo ok
#     K.clear_session()
#     cuda.select_device(0)
#     cuda.close()
    
    reset_keras(model)